In [1]:
import datetime
import gc
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import time
import warnings
from contextlib import contextmanager
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold

import lightgbm as lgb
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

import optuna
from scipy.stats import ks_2samp
from tqdm import tqdm

from lightgbm import LGBMRegressor
from boruta import BorutaPy
# import optuna.integration.lightgbm as lgb
from sklearn.preprocessing import LabelEncoder

In [2]:
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('train_final.csv')
test=pd.read_csv('test_final.csv')

In [42]:
test_id=pd.read_csv('test_final.csv')['card_id']

In [4]:
train_df=data.copy()

In [5]:
from datetime import datetime
# Fix datetime columns to be relative floating points

object_cols = train_df.select_dtypes(include=['O']).columns
ref = datetime(2017, 1, 1)

for col in object_cols:
    if 'latest' in col or 'earliest' in col:
        train_df[col] = (pd.to_datetime(train_df[col]) - ref).dt.total_seconds() / 3600.0

for col in object_cols:
    if 'latest' in col or 'earliest' in col:
        test[col] = (pd.to_datetime(test[col]) - ref).dt.total_seconds() / 3600.0
# Remove remaining features of type Object

train_df = train_df.drop(columns=train_df.select_dtypes(include=['O']).columns)
test = test.drop(columns=test.select_dtypes(include=['O']).columns)

In [6]:
train_df.head()

,feature_1,feature_2,feature_3,target,new_flag_mean,new_flag_median,new_flag_sum,new_flag_sum0,new_flag_mode,new_flag_std,...,transaction_frequency_c3_NA_1,count_c3_NA_0,latest_transaction_c3_NA_0,earliest_transaction_c3_NA_0,days_active_c3_NA_0,transaction_frequency_c3_NA_0,city_max_spent,merchant_category_max_spent,state_max_spent,subsector_max_spent
0,5,2,1,-0.820283,0.083032,0.0,23,254,False,0.276431,...,NaN,277,11603.384722,4262.302222,305,0.908197,69,560,9,34
1,4,1,0,0.392913,0.016854,0.0,6,350,False,0.128905,...,0.086957,354,10878.807222,136.495000,447,0.791946,69,307,9,34
2,2,2,0,0.688056,0.022727,0.0,1,43,False,0.150756,...,NaN,44,11585.719722,248.356111,472,0.093220,143,705,5,33
3,4,3,0,0.142495,0.083333,0.0,7,77,False,0.278045,...,0.021277,81,11339.003056,6448.372500,203,0.399015,17,278,22,37
4,1,3,0,-0.159749,0.214286,0.0,36,132,False,0.411553,...,0.000000,167,11586.840278,7560.000000,167,1.000000,17,278,22,37


In [7]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

# rmse
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns
    
# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]

    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances.png')

In [8]:
string_columns = train_df.select_dtypes(include=['object']).columns
date_columns = train_df.select_dtypes(include=['datetime64[ns]']).columns
train_df.drop(string_columns.union(date_columns), axis=1, inplace=True)

# Only keep features with na_ratio<0.4

In [9]:
na_ratio = train_df.isna().mean()

# Drop columns where the ratio of NA is greater than 0.5
train_df = train_df.loc[:, na_ratio <= 0.4]

In [10]:
feats = [fea for fea in train_df.columns if fea != "target"]

In [ ]:
# from scipy import stats
# # Calculate the Z-scores of all columns
# z_scores = stats.zscore(train_df[feats])

# # Get absolute Z-scores
# abs_z_scores = np.abs(z_scores)

# # Find rows where all the Z-scores are less than the threshold
# filtered_entries = (abs_z_scores <= 5).all(axis=1)

# # Select only the valid entries
# train_df = train_df[filtered_entries]

In [11]:
len(train_df)

201917

# Remove features stepwise

In [13]:
feature_reduction_stages = [1500, 1200, 1000, 800]

for target_feature_count in feature_reduction_stages:
    while len(train_df.drop(columns='target').columns) > target_feature_count+1:
        # Train LightGBM model
        model = lgb.LGBMRegressor(metric= 'rmse',learning_rate= 0.05)
        model.fit(train_df.drop(columns='target'), train_df['target'])

        # Get feature importances and sort them
        importances = pd.Series(model.feature_importances_, index=train_df.drop(columns='target').columns)
        sorted_features = importances.sort_values(ascending=True)

        # Drop the least important features
        features_to_drop = sorted_features.head(len(train_df.drop(columns='target').columns) - target_feature_count).index
        train_df.drop(columns=features_to_drop, inplace=True)
#         X_test.drop(columns=features_to_drop, inplace=True)

    print("Complete one round!")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 4.637589 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 279111
[LightGBM] [Info] Number of data points in the train set: 201917, number of used features: 1875
[LightGBM] [Info] Start training from score -0.393636
Complete one round!
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 3.412014 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 236884
[LightGBM] [Info] Number of data points in the train set: 201917, number of used features: 1499
[LightGBM] [Info] Start training from score -0.393636
Complete one round!
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.188220 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 199888
[LightGBM] [Info] Number of data points in the train set: 201917, 

In [14]:
model = lgb.LGBMRegressor()
model.fit(train_df.drop(columns='target'), train_df['target'])

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.484079 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 150202
[LightGBM] [Info] Number of data points in the train set: 201917, number of used features: 800
[LightGBM] [Info] Start training from score -0.393636


LGBMRegressor()

# Drop rows with z score>5

It can only based on the top20 features because too many data will ruin the kernel

In [15]:
importances = pd.Series(model.feature_importances_, index=train_df.drop(columns='target').columns)
sorted_features = importances.sort_values(ascending=True)

# Drop the least important features
features_top20 = sorted_features.tail(20).index

In [40]:
from scipy import stats
# Calculate the Z-scores of all columns
# z_scores = stats.zscore(train_df[features_top20])
df_zscore = train_df[features_top20].apply(lambda x: zscore(x, nan_policy='omit'))
# Get absolute Z-scores
abs_z_scores = np.abs(df_zscore)

# Find rows where all the Z-scores are less than the threshold
filtered_entries = ((abs_z_scores <= 10) | (pd.isna(abs_z_scores))).all(axis=1)

# Select only the valid entries
train_df = train_df[filtered_entries]

# Transform train and test

In [13]:
# test=pd.read_csv('test_final.csv')

In [60]:
# all_ele=train_df.drop(columns=['target']).columns.tolist()
# all_ele.append('card_id')

ytrain=train_df['target']
xtrain=train_df.drop(columns=['target'])

In [61]:
xtest=test[train_df.drop(columns=['target']).columns.tolist()].copy()

In [62]:
xtrain['feature_3']=data['feature_3']
xtest['feature_3']=test['feature_3']

In [63]:
categorical_feats = ['feature_1', 'feature_2', 'feature_3']

for col in categorical_feats:
    print(col)
    lbl = LabelEncoder()
    lbl.fit(list(xtrain[col].values.astype('str')) + list(xtest[col].values.astype('str')))
    xtrain[col] = lbl.transform(list(xtrain[col].values.astype('str')))
    xtest[col] = lbl.transform(list(xtest[col].values.astype('str')))

feature_1
feature_2
feature_3


In [64]:
df_all = pd.concat([xtrain, xtest])
df_all = pd.get_dummies(df_all, columns=categorical_feats)

len_train = xtrain.shape[0]

xtrain = df_all[:len_train]
xtest = df_all[len_train:]

In [20]:
# prepare for modeling
# id_train = xtrain['card_id'].copy()
# xtrain.drop('card_id', axis = 1, inplace = True)
# id_test = xtest['card_id'].copy()
# xtest.drop('card_id', axis = 1, inplace = True)

# Use optuna for choosing params

In [65]:
import optuna
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

# Define the objective function
def objective(trial):
    
    X_train, X_test, y_train, y_test = train_test_split(xtrain, ytrain, test_size=0.2)

    
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'verbose':-1,
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
    }

    # Train the model
    gbm = lgb.LGBMRegressor(**param)
    gbm.fit(X_train, y_train, eval_set=[(X_test, y_test)])

    # Predict and calculate RMSE
    preds = gbm.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse

# Create a study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)  # You can change the number of trials

# Best parameters
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2023-12-04 20:02:25,618] A new study created in memory with name: no-name-1fe3fd42-b00e-4592-8c84-4572825ca141
[I 2023-12-04 20:03:01,453] Trial 0 finished with value: 3.7402987846976603 and parameters: {'lambda_l1': 0.014482608322364938, 'lambda_l2': 1.4687639055370897e-07, 'num_leaves': 95, 'feature_fraction': 0.5664443993970065, 'bagging_fraction': 0.6814234346626591, 'bagging_freq': 7, 'min_child_samples': 85, 'learning_rate': 0.04518047205173545}. Best is trial 0 with value: 3.7402987846976603.
[I 2023-12-04 20:03:29,721] Trial 1 finished with value: 3.6566559812388726 and parameters: {'lambda_l1': 6.951427779648484e-08, 'lambda_l2': 6.332153527475398e-08, 'num_leaves': 91, 'feature_fraction': 0.42228588141923534, 'bagging_fraction': 0.7044968326422897, 'bagging_freq': 3, 'min_child_samples': 25, 'learning_rate': 0.030557715251669917}. Best is trial 1 with value: 3.6566559812388726.
[I 2023-12-04 20:04:04,701] Trial 2 finished with value: 3.6406438887729515 and parameters: {'la

[I 2023-12-04 20:18:55,510] Trial 21 finished with value: 3.6532013144543254 and parameters: {'lambda_l1': 9.752599366270442, 'lambda_l2': 0.0001488370021664347, 'num_leaves': 121, 'feature_fraction': 0.9037676917284035, 'bagging_fraction': 0.8895788757114175, 'bagging_freq': 6, 'min_child_samples': 66, 'learning_rate': 0.06310830088982684}. Best is trial 5 with value: 3.5996662426793193.
[I 2023-12-04 20:19:49,173] Trial 22 finished with value: 3.654223577663399 and parameters: {'lambda_l1': 0.07848628741332174, 'lambda_l2': 0.00012585922636568493, 'num_leaves': 176, 'feature_fraction': 0.78481702789697, 'bagging_fraction': 0.8741525374075746, 'bagging_freq': 2, 'min_child_samples': 80, 'learning_rate': 0.05241006478937153}. Best is trial 5 with value: 3.5996662426793193.
[I 2023-12-04 20:20:40,159] Trial 23 finished with value: 3.542113849338635 and parameters: {'lambda_l1': 0.009378943867781906, 'lambda_l2': 1.6036090134055939e-06, 'num_leaves': 144, 'feature_fraction': 0.8952864576

[I 2023-12-04 20:37:18,324] Trial 42 finished with value: 3.603187712183367 and parameters: {'lambda_l1': 8.650310911236746, 'lambda_l2': 5.310971366791875e-07, 'num_leaves': 114, 'feature_fraction': 0.9220246634592111, 'bagging_fraction': 0.9616268064600636, 'bagging_freq': 5, 'min_child_samples': 92, 'learning_rate': 0.026076157600295503}. Best is trial 23 with value: 3.542113849338635.
[I 2023-12-04 20:38:09,939] Trial 43 finished with value: 3.6800902165903233 and parameters: {'lambda_l1': 6.636351489613675, 'lambda_l2': 1.6896363033390012e-06, 'num_leaves': 120, 'feature_fraction': 0.927106545418231, 'bagging_fraction': 0.8997631951438317, 'bagging_freq': 3, 'min_child_samples': 87, 'learning_rate': 0.020264017813497168}. Best is trial 23 with value: 3.542113849338635.
[I 2023-12-04 20:38:54,016] Trial 44 finished with value: 3.7010870554959734 and parameters: {'lambda_l1': 0.6904519912544396, 'lambda_l2': 4.749811378002748e-07, 'num_leaves': 74, 'feature_fraction': 0.981165877174

Number of finished trials:  50
Best trial:
  Value:  3.542113849338635
  Params: 
    lambda_l1: 0.009378943867781906
    lambda_l2: 1.6036090134055939e-06
    num_leaves: 144
    feature_fraction: 0.8952864576742605
    bagging_fraction: 0.9284567661492465
    bagging_freq: 5
    min_child_samples: 49
    learning_rate: 0.07958940675014119


In [66]:
best_params=trial.params

# Train cv lightgbm and predict

In [67]:
def modeling_lgbm_cross_validation(params, X, y, nr_folds=5):
    clfs = list()
    oof_preds = np.zeros(X.shape[0])

    kfolds = StratifiedKFold(n_splits=nr_folds, shuffle=True, random_state=42)
    
    kfolds = KFold(n_splits=nr_folds, shuffle=True, random_state=42)
    for n_fold, (trn_idx, val_idx) in enumerate(kfolds.split(X, y)):

        X_train, y_train = X.iloc[trn_idx], y.iloc[trn_idx]
        X_valid, y_valid = X.iloc[val_idx], y.iloc[val_idx]
        
        params['early_stopping_rounds']=500
        model = lgb.LGBMRegressor(**params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_valid, y_valid)],
            eval_metric='rmse'
        
        )

        clfs.append(model)
        oof_preds[val_idx] = model.predict(X_valid, num_iteration=model.best_iteration_)

        del X_train, y_train, X_valid, y_valid
        gc.collect()  
    score = mean_squared_error(y, oof_preds) ** .5
    return clfs, score

def predict_cross_validation(test, clfs, ntree_limit=None):
    sub_preds = np.zeros(test.shape[0])
    for i, model in enumerate(clfs, 1):

        num_tree = 10000
        if not ntree_limit:
            ntree_limit = num_tree

        if isinstance(model, lgb.sklearn.LGBMRegressor):
            if model.best_iteration_:
                num_tree = min(ntree_limit, model.best_iteration_)

            test_preds = model.predict(test, raw_score=True, num_iteration=num_tree)

        sub_preds += test_preds

    sub_preds = sub_preds / len(clfs)
    ret = pd.Series(sub_preds, index=test.index)
    ret.index.name = test.index.name
    return ret

In [68]:
with timer("Run LightGBM with kfold"):
    # modeling
    nr_folds=10
    clfs, score = modeling_lgbm_cross_validation(best_params,
                                                    xtrain,
                                                    ytrain,
                                                    nr_folds
                                                    )


    pred_test = predict_cross_validation(xtest, clfs)

[LightGBM] [Warning] feature_fraction is set=0.8952864576742605, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8952864576742605
[LightGBM] [Warning] lambda_l1 is set=0.009378943867781906, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.009378943867781906
[LightGBM] [Warning] lambda_l2 is set=1.6036090134055939e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.6036090134055939e-06
[LightGBM] [Warning] bagging_fraction is set=0.9284567661492465, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9284567661492465
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] early_stopping_round is set=500, early_stopping_rounds=500 will be ignored. Current value: early_stopping_round=500
[LightGBM] [Warning] feature_fraction is set=0.8952864576742605, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8952864576742605
[LightGBM] [Warning] 

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.152303 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 149598
[LightGBM] [Info] Number of data points in the train set: 180076, number of used features: 808
[LightGBM] [Warning] feature_fraction is set=0.8952864576742605, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8952864576742605
[LightGBM] [Warning] lambda_l1 is set=0.009378943867781906, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.009378943867781906
[LightGBM] [Warning] lambda_l2 is set=1.6036090134055939e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.6036090134055939e-06
[LightGBM] [Warning] bagging_fraction is set=0.9284567661492465, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9284567661492465
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning

Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[38]	valid_0's rmse: 3.70697	valid_0's l2: 13.7416
[LightGBM] [Warning] feature_fraction is set=0.8952864576742605, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8952864576742605
[LightGBM] [Warning] lambda_l1 is set=0.009378943867781906, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.009378943867781906
[LightGBM] [Warning] lambda_l2 is set=1.6036090134055939e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.6036090134055939e-06
[LightGBM] [Warning] bagging_fraction is set=0.9284567661492465, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9284567661492465
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8952864576742605, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.89528645767426

[LightGBM] [Warning] feature_fraction is set=0.8952864576742605, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8952864576742605
[LightGBM] [Warning] lambda_l1 is set=0.009378943867781906, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.009378943867781906
[LightGBM] [Warning] lambda_l2 is set=1.6036090134055939e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.6036090134055939e-06
[LightGBM] [Warning] bagging_fraction is set=0.9284567661492465, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9284567661492465
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.181143 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 149460
[LightGBM] [Info] Number of data points in the train set: 180077, number of used features: 808
[LightGBM] [Warning

[LightGBM] [Info] Start training from score -0.380064
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[36]	valid_0's rmse: 3.59979	valid_0's l2: 12.9585
[LightGBM] [Warning] feature_fraction is set=0.8952864576742605, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8952864576742605
[LightGBM] [Warning] lambda_l1 is set=0.009378943867781906, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.009378943867781906
[LightGBM] [Warning] lambda_l2 is set=1.6036090134055939e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.6036090134055939e-06
[LightGBM] [Warning] bagging_fraction is set=0.9284567661492465, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9284567661492465
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8952864576742605, colsample_bytree=1.0 will be ign

In [70]:
print('Score for this model is',score)

Score for this model is 3.659622489141572


In [69]:
pd.DataFrame({'card_id':test['card_id'].tolist(),'target':pred_test}).to_csv('pred_test_1.csv',index=True)